In [57]:
import pandas as pd
from sklearn.metrics.pairwise import linear_kernel
from sklearn.feature_extraction.text import TfidfVectorizer

In [116]:
data = pd.read_csv('C:/Users/USER/Desktop/MOHIM/PFA/ScrapeStorm/Complet/HardDataSet.csv',low_memory=False,encoding='latin-1')

In [117]:
print(data.columns.tolist())

['Title', 'Company_Name', 'Skills', 'Country']


In [118]:
list_companies = data['Company_Name']
list_jobs = data['Title']

In [119]:
len(list_jobs)

249

In [120]:
for country in list_jobs:
    print(country)

web developer
web developer
web developer
web developer
web developer
web developer
web developer
web developer
web developer
web developer
web developer
web developer
web developer
web developer
web developer
web developer
web developer
web developer
web developer
web developer
web developer
web developer
web developer
web developer
web developer
web developer
web developer
web developer
web developer
web developer
web developer
web developer
web developer
web developer
web developer
web developer
web developer
web developer
web developer
web developer
web developer
web developer
web developer
web developer
web developer
web developer
web developer
web developer
Data scientist
Data scientist
Data scientist
Data scientist
Data scientist
Data scientist
Data scientist
Data scientist
Data scientist
Data scientist
Data scientist
Data scientist
Data scientist
Data scientist
Data scientist
Data scientist
Data scientist
Data scientist
Data scientist
Data scientist
Data scientist
Data scientis

In [121]:
def get_unique_jobs(list_jobs):
    unique_jobs = []
    for job in list_jobs:
        if(job not in unique_jobs):
            unique_jobs.append(job)
    return unique_jobs


In [122]:
def get_score_job(job,vector_user_job,vectorizer):
    vector_unique_job = vectorizer.transform(job)
    list_similarity = linear_kernel(vector_user_job,vector_unique_job).flatten()
    score_similarity = sum(list_similarity)/len(list_similarity)
    return score_similarity,job
    

In [123]:
def get_similar_jobs(user_job,unique_jobs):

    vectorizer = TfidfVectorizer(stop_words='english', use_idf=True, smooth_idf=True)
    vectorizer.fit(unique_jobs)
    list_general = []
    #list_general = [[]]
    #list_general.append([])
    vector_user_job = vectorizer.transform(user_job)
    for job in unique_jobs:
        job = [job]
        #list_general[1].append(job)
        #vector_unique_job = vectorizer.transform(job)
        #list_similarity = linear_kernel(vector_user_job,vector_unique_job).flatten()
        #score_similarity = sum(list_similarity)/len(list_similarity)
        #list_general[0].append(score_similarity)
        list_general.append(get_score_job(job,vector_user_job,vectorizer))
    return list_general

In [124]:
unique_jobs = []
unique_jobs = get_unique_jobs(list_jobs)

In [125]:
for job in unique_jobs:
    print(job)

web developer
Data scientist
Security engineer
Software engineer
Software engineer in Test
BI consultant


In [126]:
user_job = "back end developer"
user_job = [user_job]
general = get_similar_jobs(user_job,unique_jobs)
#job_simalarity_score = sorted(general[0], reverse = True)
general = sorted(general, reverse = True)
#sorted(general, key=lambda student: general[0])
print(general[0][1])

    

['web developer']
